In [2]:
from Utils.loadset import getDataSet
from tensorflow.keras import Sequential, Model
from tensorflow.keras.layers import *
from tensorflow.keras.optimizers import Adam
import tensorflow as tf
import tensorflow_probability as tfp
tfk = tf.keras
tfkl = tf.keras.layers
tfpl = tfp.layers
tfd = tfp.distributions
import os
from trainer import Trainer
try:
    from Utils.connection_cfg import *
except Exception as e:
    PSWD = None
    USRN = None
    
from Utils.Data import dataWrapper
from Utils.transform import ToCategorical, cutOut

Num GPUs Available: 1


Using TensorFlow backend.


In [3]:
dimension = (64,64)
batch_size = 60
channels = 5
optimizer = Adam( lr = 1e-3 )
slices = [256,320,256,320]
cutOutFrame = cutOut(slices)

categorical_list = [0,1,5,10,15,30,60,120]
categorical = ToCategorical(categorical_list)

PRETRAINING_TRANSFORMATIONS = [cutOutFrame]
TRANSFORMATIONS = [categorical]
#TRANSFORMATIONS = None

def NLL(y_true, y_hat):
    return -y_hat.log_prob(y_true)


def provideData(flatten=False,dimension=dimension,batch_size=60,transform=None,preTransformation=None):

    getDataSet(DatasetFolder,year=[2017],username=USRN,pswd=PSWD)
    train,test = dataWrapper(PathToData,
                            dimension=dimension,
                            channels=channels,
                            batch_size=batch_size,
                            overwritecsv=True,
                            flatten=flatten,
                            onlyUseYears=[2017],
                            transform=transform,
                            preTransformation=preTransformation)
    
    return train,test
DatasetFolder = "./Data/RAW"
PathToData = os.path.join(DatasetFolder,"MonthPNGData")

train, test = provideData(dimension=dimension,
                          batch_size=batch_size,
                          transform=TRANSFORMATIONS,
                          preTransformation=PRETRAINING_TRANSFORMATIONS)

Found Year :  2017 => won't download this year again... please check for consistency
Finished Loading Dataset
 


In [4]:
input_shape = (64,64,1)
encoded_size = 16
base_depth = 32

prior = tfd.Independent(tfd.Normal(loc=tf.zeros(encoded_size), scale=1),
                        reinterpreted_batch_ndims=1)

encoder = tfk.Sequential([
    tfkl.InputLayer(input_shape=input_shape),
    tfkl.Lambda(lambda x: tf.cast(x, tf.float32) - 0.5),
    tfkl.Conv2D(base_depth, 5, strides=1,
                padding='same', activation=tf.nn.leaky_relu),
    tfkl.Conv2D(base_depth, 5, strides=2,
                padding='same', activation=tf.nn.leaky_relu),
    tfkl.Conv2D(2 * base_depth, 5, strides=1,
                padding='same', activation=tf.nn.leaky_relu),
    tfkl.Conv2D(2 * base_depth, 5, strides=2,
                padding='same', activation=tf.nn.leaky_relu),
    tfkl.Conv2D(4 * encoded_size, 7, strides=1,
                padding='valid', activation=tf.nn.leaky_relu),
    tfkl.Flatten(),
    tfkl.Dense(tfpl.MultivariateNormalTriL.params_size(encoded_size),
               activation=None),
    tfpl.MultivariateNormalTriL(
        encoded_size,
        activity_regularizer=tfpl.KLDivergenceRegularizer(prior)),
])


decoder = tfk.Sequential([
    tfkl.InputLayer(input_shape=[encoded_size]),
    tfkl.Reshape([1, 1, encoded_size]),
    tfkl.Conv2DTranspose(2 * base_depth, 7, strides=1,
                         padding='valid', activation=tf.nn.leaky_relu),
    tfkl.Conv2DTranspose(2 * base_depth, 5, strides=1,
                         padding='same', activation=tf.nn.leaky_relu),
    tfkl.Conv2DTranspose(2 * base_depth, 5, strides=2,
                         padding='same', activation=tf.nn.leaky_relu),
    tfkl.Conv2DTranspose(base_depth, 5, strides=1,
                         padding='same', activation=tf.nn.leaky_relu),
    tfkl.Conv2DTranspose(base_depth, 5, strides=2,
                         padding='same', activation=tf.nn.leaky_relu),
    tfkl.Conv2DTranspose(base_depth, 5, strides=1,
                         padding='same', activation=tf.nn.leaky_relu),
    tfkl.Conv2D(filters=1, kernel_size=5, strides=1,
                padding='same', activation=None),
    tfkl.Flatten(),
     tfpl.IndependentBernoulli(input_shape, tfd.Bernoulli.logits),
])

vae = tfk.Model(inputs=encoder.inputs,
                outputs=decoder(encoder.outputs[0]))

Instructions for updating:
Do not pass `graph_parents`.  They will  no longer be used.


In [5]:
vae.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 64, 64, 1)]       0         
_________________________________________________________________
lambda (Lambda)              (None, 64, 64, 1)         0         
_________________________________________________________________
conv2d (Conv2D)              (None, 64, 64, 32)        832       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 32, 32, 32)        25632     
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 32, 32, 64)        51264     
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 16, 16, 64)        102464    
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 10, 10, 64)        200768

In [1]:
from Utils.loadset import getDataSet
from tensorflow.keras import Sequential, Model
from tensorflow.keras.layers import *
from tensorflow.keras.optimizers import Adam
import tensorflow as tf
import tensorflow_probability as tfp


tfk = tf.keras
tfkl = tf.keras.layers
tfpl = tfp.layers
tfd = tfp.distributions
import os
from trainer import Trainer
try:
    from Utils.connection_cfg import *
except Exception as e:
    PSWD = None
    USRN = None
    
from Utils.Data import dataWrapper
from Utils.transform import ToCategorical, cutOut


Num GPUs Available: 1


Using TensorFlow backend.


In [2]:
dimension = (64,64)
batch_size = 400
channels = 5
optimizer = Adam( lr = 1e-4 )
slices = [256,320,256,320]
cutOutFrame = cutOut(slices)

categorical_list = [0,1,5,10,15,30,60,120]
categorical = ToCategorical(categorical_list)

PRETRAINING_TRANSFORMATIONS = [cutOutFrame]
TRANSFORMATIONS = [categorical]
#TRANSFORMATIONS = None

def NLL(y_true, y_hat):
    return -y_hat.log_prob(y_true)


def provideData(flatten=False,dimension=dimension,batch_size=60,transform=None,preTransformation=None):

    getDataSet(DatasetFolder,year=[2017],username=USRN,pswd=PSWD)
    train,test = dataWrapper(PathToData,
                            dimension=dimension,
                            channels=channels,
                            batch_size=batch_size,
                            overwritecsv=True,
                            flatten=flatten,
                            onlyUseYears=[2017],
                            transform=transform,
                            preTransformation=preTransformation)
    
    return train,test
DatasetFolder = "./Data/RAW"
PathToData = os.path.join(DatasetFolder,"MonthPNGData")

train, test = provideData(dimension=dimension,
                          batch_size=batch_size,
                          transform=TRANSFORMATIONS,
                          preTransformation=PRETRAINING_TRANSFORMATIONS)

Found Year :  2017 => won't download this year again... please check for consistency
Finished Loading Dataset
 


In [4]:
print(len(train))
print(len(train[0][0]))
print(len(train[0][1]))
print(len(train.listOfFiles))

for i,(x,y) in enumerate(train):
    print("\t",i,"\n------------------\n")
    continue

197
LEN indizes:  78830
id_list:  400 0 400
400
LEN indizes:  78830
id_list:  400 0 400
400
78840
LEN indizes:  78830
id_list:  400 0 400
	 0 
------------------

LEN indizes:  78830
id_list:  400 400 800
	 1 
------------------

LEN indizes:  78830
id_list:  400 800 1200
	 2 
------------------

LEN indizes:  78830
id_list:  400 1200 1600
	 3 
------------------

LEN indizes:  78830
id_list:  400 1600 2000
	 4 
------------------

LEN indizes:  78830
id_list:  400 2000 2400
	 5 
------------------

LEN indizes:  78830
id_list:  400 2400 2800
	 6 
------------------

LEN indizes:  78830
id_list:  400 2800 3200
	 7 
------------------

LEN indizes:  78830
id_list:  400 3200 3600
	 8 
------------------

LEN indizes:  78830
id_list:  400 3600 4000
	 9 
------------------

LEN indizes:  78830
id_list:  400 4000 4400
	 10 
------------------

LEN indizes:  78830
id_list:  400 4400 4800
	 11 
------------------

LEN indizes:  78830
id_list:  400 4800 5200
	 12 
------------------

LEN indiz

	 113 
------------------

LEN indizes:  78830
id_list:  400 45600 46000
	 114 
------------------

LEN indizes:  78830
id_list:  400 46000 46400
	 115 
------------------

LEN indizes:  78830
id_list:  400 46400 46800
	 116 
------------------

LEN indizes:  78830
id_list:  400 46800 47200
	 117 
------------------

LEN indizes:  78830
id_list:  400 47200 47600
	 118 
------------------

LEN indizes:  78830
id_list:  400 47600 48000
	 119 
------------------

LEN indizes:  78830
id_list:  400 48000 48400
	 120 
------------------

LEN indizes:  78830
id_list:  400 48400 48800
	 121 
------------------

LEN indizes:  78830
id_list:  400 48800 49200
	 122 
------------------

LEN indizes:  78830
id_list:  400 49200 49600
	 123 
------------------

LEN indizes:  78830
id_list:  400 49600 50000
	 124 
------------------

LEN indizes:  78830
id_list:  400 50000 50400
	 125 
------------------

LEN indizes:  78830
id_list:  400 50400 50800
	 126 
------------------

LEN indizes:  78830
id_l